# Data Literacy Project
#### University of Tübingen, Winter Term 2021/22

### Testing 01
---

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
apiResponse = requests.get('https://de.openparliament.tv/api/v1/electoralPeriod/DE-019')
data = json.loads(apiResponse.text)
data = pd.json_normalize(data, ['data','relationships','sessions','data'])
data

In [95]:
base_request = 'https://de.openparliament.tv/api/v1/search/media/?'
max_page = 2

apiResponse = requests.get(base_request + 'page=' + str(1)).json()
result = apiResponse['data']
for page in range(2, max_page+1):
    apiResponse = requests.get(base_request + 'page=' + str(page)).json()
    result.extend(apiResponse['data'])
    
data = pd.json_normalize(result)
# data to store
# - relationships.people.data.attributes.label                           (main-speaker)
# - relationships.people.data.attributes.party.label                     (main-speaker party)
# - attributes.TextContent.DICT(textBody.type, textBody.sentences.sentences.text) (speech and comments)

,type,id,_score,_highlight,_finds,attributes.originID,attributes.originMediaID,attributes.creator,attributes.license,attributes.parliament,...,relationships.agendaItem.data.links.self,relationships.documents.data,relationships.documents.links.self,relationships.organisations.data,relationships.organisations.links.self,relationships.terms.data,relationships.terms.links.self,relationships.people.data,relationships.people.links.self,relationships.annotations.links.self
0,media,DE-0190001002,0,None,None,None,7164718,Deutscher Bundestag,"<a href=""https://www.bundestag.de/nutzungsbedi...",DE,...,https://de.openparliament.tv/api/v1/agendaItem...,[],https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'person', 'id': 'Q70407', 'attribute...",https://de.openparliament.tv/api/v1/search/ann...,https://de.openparliament.tv/api/v1/search/ann...
1,media,DE-0190001007,0,None,None,None,7164730,Deutscher Bundestag,"<a href=""https://www.bundestag.de/nutzungsbedi...",DE,...,https://de.openparliament.tv/api/v1/agendaItem...,"[{'type': 'document', 'id': '4022', 'attribute...",https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'organisation', 'id': 'Q1826856', 'a...",https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'person', 'id': 'Q1599545', 'attribu...",https://de.openparliament.tv/api/v1/search/ann...,https://de.openparliament.tv/api/v1/search/ann...
2,media,DE-0190001008,0,None,None,None,7164731,Deutscher Bundestag,"<a href=""https://www.bundestag.de/nutzungsbedi...",DE,...,https://de.openparliament.tv/api/v1/agendaItem...,"[{'type': 'document', 'id': '4022', 'attribute...",https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'organisation', 'id': 'Q1023134', 'a...",https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'person', 'id': 'Q1770968', 'attribu...",https://de.openparliament.tv/api/v1/search/ann...,https://de.openparliament.tv/api/v1/search/ann...
3,media,DE-0190001010,0,None,None,None,7164735,Deutscher Bundestag,"<a href=""https://www.bundestag.de/nutzungsbedi...",DE,...,https://de.openparliament.tv/api/v1/agendaItem...,"[{'type': 'document', 'id': '4022', 'attribute...",https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'organisation', 'id': 'Q1007353', 'a...",https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'person', 'id': 'Q920726', 'attribut...",https://de.openparliament.tv/api/v1/search/ann...,https://de.openparliament.tv/api/v1/search/ann...
4,media,DE-0190001011,0,None,None,None,7164736,Deutscher Bundestag,"<a href=""https://www.bundestag.de/nutzungsbedi...",DE,...,https://de.openparliament.tv/api/v1/agendaItem...,"[{'type': 'document', 'id': '4022', 'attribute...",https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'person', 'id': 'Q70407', 'attribute...",https://de.openparliament.tv/api/v1/search/ann...,https://de.openparliament.tv/api/v1/search/ann...
5,media,DE-0190001014,0,None,None,None,7164755,Deutscher Bundestag,"<a href=""https://www.bundestag.de/nutzungsbedi...",DE,...,https://de.openparliament.tv/api/v1/agendaItem...,"[{'type': 'document', 'id': '4023', 'attribute...",https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,[],https://de.openparliament.tv/api/v1/search/ann...,"[{'type': 'person', 'id': 'Q16019', 'attribute...",https://de.openparliament.tv/api/v1/search/ann...,https://de.openparliament.tv/api/v1/search/ann...
6,media,DE-0190002004,0,None,None,None,7174011,Deutscher Bundestag,"<a href=""https://www.bundestag.de/nutzungsbedi...",DE,...,https://de.openparliament.tv/api/v1/agendaItem...,"[{'type': 'document', 'id': '40

In [119]:
dataExtracted = data[['attributes.textContents', 'relationships.people.data']]

dataExtracted['sentences'] = dataExtracted.apply(lambda dataRow : getDictOfTextContents(dataRow['attributes.textContents']), axis=1)
dataExtracted = dataExtracted.drop(columns=['attributes.textContents'])

dataExtracted['main-speaker'] = dataExtracted.apply(lambda dataRow : dataRow['relationships.people.data'][0]['attributes']['label'], axis=1)
dataExtracted['main-speaker-party'] = dataExtracted.apply(lambda dataRow : dataRow['relationships.people.data'][0]['attributes']['party']['label'], axis=1)
dataExtracted = dataExtracted.drop(columns=['relationships.people.data'])


dataExtracted.loc[1].sentences

/var/folders/tl/h6dzm0t54f9b2m4r_wdf4vwm0000gn/T/ipykernel_3740/3707124614.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataExtracted['sentences'] = dataExtracted.apply(lambda dataRow : getDictOfTextContents(dataRow['attributes.textContents']), axis=1)


[{'speech': 'Herr Präsident! Liebe Kolleginnen und Kollegen! Sehr geehrte Damen und Herren! Bei aller Kritik an der parlamentarischen Demokratie, der Geschäftsordnung und vielem anderen mehr will ich zumindest an eines erinnern: Wir sitzen hier aufgrund von freien Wahlen. Das haben Millionen andere nicht.'},
 {'comment': '(Beifall bei der LINKEN, der SPD und dem BÜNDNIS\xa090/DIE GRÜNEN sowie bei Abgeordneten der CDU/CSU, der AfD und der FDP)'},
 {'speech': 'Und wir sind dem Grundgesetz verpflichtet, das nicht einfach irgendein Gesetz ist, sondern das die humane und demokratische Antwort auf die Verheerung und die Leichenberge des NS-Faschismus gewesen ist. Das gilt es jeden Tag zu verteidigen.'},
 {'comment': '(Beifall bei der LINKEN sowie bei Abgeordneten der SPD, der FDP und des BÜNDNISSES\xa090/DIE GRÜNEN)'},
 {'speech': 'Aber es gibt natürlich auch berechtigte Kritik an unserem Verfahren hier, am Parlamentarismus. Dieser müssen wir uns stellen, und wir müssen das bessern. Deswegen

In [ ]:
data['sentences'] = None
data['main-speaker'] = None
data['main-speaker-party'] = None

In [94]:
def getDictOfTextContents(textContents):
    body = textContents[0]['textBody']
    subBody = []
    for j in range(len(body)):
        textPart = body[j]
        subBody.append({textPart['type'] : 
                        ' '.join([s['text'] for s in textPart['sentences']])})
    return subBody

